In [21]:

# general settings
basePath = './'
filename = 'umap_on_emotion_scores'
figdir = '%sfig/' %(basePath)
savdir = '%sres/%s/' %(basePath,filename)


#!pip install umap-lear
import os
import numpy as np
import pandas as pd
import umap
import hdf5storage as h5
from datetime import datetime
import scipy.io as sio
from scipy.spatial import distance
import matplotlib.pyplot as plt
%matplotlib inline

if os.path.exists(savdir) == 0:
    os.makedirs(savdir)


umap.UMAP()

UMAP(a=None, angular_rp_forest=False, b=None, init='spectral',
   learning_rate=1.0, local_connectivity=1.0, metric='euclidean',
   metric_kwds=None, min_dist=0.1, n_components=2, n_epochs=None,
   n_neighbors=15, negative_sample_rate=5, random_state=None,
   repulsion_strength=1.0, set_op_mix_ratio=1.0, spread=1.0,
   target_metric='categorical', target_metric_kwds=None,
   target_n_neighbors=-1, target_weight=0.5, transform_queue_size=4.0,
   transform_seed=42, verbose=False)

In [ ]:
# start analysis
sbjIDs = ['Subject1','Subject2','Subject3','Subject4','Subject5']

# UMAP parameters
n_neighbour = 10;
min_dist = 1.0;
seed = 1;
spread = 3;

for sbjID in sbjIDs:
    datapath = '%sres/emotion2020_analysis_BATCH//category/%s/nCV6_mROIDec//summary/res_mROIs_feat.mat' %(basePath,sbjID)
    
    if os.path.exists(savdir) == 0:
        print "Decoded features for %s were not yet prepared." %(sbjID)
    else:
        data = h5.loadmat(datapath)

        saveFname    = '%snn%d_md%.1f_sp%d_seed%d_%s.mat' %(savdir,n_neighbour,min_dist,spread,seed,sbjID)
        saveFnameChk = '%snn%d_md%.1f_sp%d_seed%d_%s_log.log' %(savdir,n_neighbour,min_dist,spread,seed,sbjID)
        
        if os.path.exists(saveFnameChk) == 1:
            print('Skip:%s' %(saveFnameChk))
        else:
            print('Start:%s' %(saveFnameChk))
            d = datetime.today()
            print('Start: %s:' %(d))
            ff = open(saveFnameChk,'w')
            ff.write(' ')
            try:
                ff.close()
            except:
                print 'Failed. close ' + saveFnameChk


            feattrue = data['feat_true']
            featpred = data['feat_decoded']

            # estimate mapping function
            cat_umap_ = umap.UMAP(n_neighbors=n_neighbour,
                                  n_components=2, metric='correlation', 
                                  random_state=seed, 
                                  min_dist = min_dist,
                                  spread = spread
                                 ).fit(feattrue)
            cat_umap = cat_umap_.embedding_

            # transform decoded scores
            d1 = cat_umap_.transform(featpred[0:1,:])
            drest = cat_umap_.transform(featpred[1:,:])
            dec_umap = np.concatenate([d1, drest])

            print saveFname
            sio.savemat(saveFname,{'cat_umap':cat_umap,'dec_umap':dec_umap})

            plt.figure(figsize=(5,5))
            plt.scatter(cat_umap[:, 0], cat_umap[:, 1], s= 5,c='black')
            plt.scatter(dec_umap[:, 0], dec_umap[:, 1], s= 5,c='red')
            plt.axis('image')
            plt.show()





